## LIME と SHAP によるデータセット分析

In [11]:
import pandas as pd
import python.data_manager as dm
import importlib

In [12]:
importlib.reload(dm)

2024-05-21 15:11:49,799 : python.data_manager : INFO : 25 : Test_message


<module 'python.data_manager' from 'c:\\Users\\zigza\\OneDrive\\ドキュメント\\0_大学院\\Projects\\Test_LIME_SHAP\\src\\python\\data_manager.py'>

In [13]:
left_df = dm.read_folder("../data/Reflection/")
right_df = dm.read_folder("../data/Grade/")

2024-05-21 15:11:49,816 : python.data_manager : INFO : 47 : read ..\data\Reflection *.csv data...
2024-05-21 15:11:49,819 : python.data_manager : INFO : 50 : read : [WindowsPath('../data/Reflection/1.csv'), WindowsPath('../data/Reflection/test.csv')]
2024-05-21 15:11:49,824 : python.data_manager : INFO : 57 : get 4 rows.
2024-05-21 15:11:49,827 : python.data_manager : INFO : 47 : read ..\data\Grade *.csv data...
2024-05-21 15:11:49,827 : python.data_manager : INFO : 50 : read : [WindowsPath('../data/Grade/gradetesst.csv')]
2024-05-21 15:11:49,831 : python.data_manager : INFO : 57 : get 3 rows.


In [14]:
left_df.head()

,userid,data
0,user1,aaaaaaaaaaaaaaaaa
1,user2,ddddddddddddddddddd
2,user 22,bbbbbbbbbbbbbbbbbbb
3,user1,aaaaaaaaaaaaaaaaaaa


In [15]:
key = "userid"
df = pd.merge(left=left_df, right=right_df, on=key)

In [16]:
df.head()

,userid,data,grade
0,user1,aaaaaaaaaaaaaaaaa,A
1,user1,aaaaaaaaaaaaaaaaaaa,A
2,user2,ddddddddddddddddddd,F
3,user 22,bbbbbbbbbbbbbbbbbbb,C


In [ ]:
train, vaild, test = dm.sprit_data_for_user(df, key=key, label="grade")